In [13]:
import pandas as pd

action_est = pd.read_csv("../result/action_estimation.csv")
emotion_est = pd.read_csv("../result/emtion_estimation.csv")

In [14]:
action_est

,action,efficacy
0,명상,"[0.14293085, 0.17040543, 0.22714882, 0.21609108]"
1,운동,"[0.14634378254413605, 0.13446630785862604, 0.1..."
2,산책,"[0.13834033906459808, 0.07066643610596657, 0.0..."
3,수면,"[0.14381973445415497, 0.08278455585241318, 0.1..."
4,독서,"[0.18120643496513367, 0.09683062881231308, 0.0..."


In [15]:
emotion_est

,감정,추정치
0,행복,"[0.5415818492571512, 0.5508736968040466, 0.467..."
1,열정,"[0.5338602662086487, 0.6573270757993063, 0.558..."
2,두려움,"[0.49721041321754456, 0.7298627297083536, 0.77..."
3,분노,"[0.519708514213562, 0.623987098534902, 0.77443..."
4,우울,"[0.4931264321009318, 0.6200025081634521, 1.0, ..."
5,슬픔,"[0.4928437868754069, 0.6017196377118429, 0.607..."


In [23]:
type(emotion_est['추정치'][0])

str